In [47]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [48]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [49]:
from autogen import AssistantAgent,GroupChat,GroupChatManager,UserProxyAgent

#agent1

user_agent=UserProxyAgent(
    human_input_mode="NEVER",
    name="user_agent",
    code_execution_config={
        "use_docker":False
    }
)
#agent 2

interviewer_agent=AssistantAgent(
    name="interviewer_agent",
    human_input_mode="NEVER",
     llm_config=llm_config,
    system_message="""
    You are an experienced technical interviewer evaluating candidates for Python development roles.  
    Your task is to assess their ability to write clear, efficient, and correct code.
    
    Prompt the candidate with this question:
    
    "Write a Python function that converts a list into a dictionary. Each element of the list should be used as a key in the dictionary, and the value can be the index of the element in the list."
    
    Expect the candidate to write a function that:
    - Accepts a list as input
    - Returns a dictionary with list items as keys and their indices as values
    - Handles edge cases like empty lists or duplicates (optional depending on candidate level)
    
    Once the candidate submits the code, review it for:
    - Correctness
    - Use of appropriate Python data structures
    - Readability and clean coding practices
    - Optional: Time and space efficiency for large lists

     """
)
#agent 3

answer_agent=AssistantAgent(
    name="answer_agent",
     llm_config=llm_config,
    system_message="""
    You are a helpful and precise answer agent that takes Python code from an interviewer agent and executes it to display the result.
    
    Your job is to:
    1. Safely run the Python code provided.
    2. Display the output clearly.
    3. Explain the result briefly if needed.
    
    You are responding to this instruction:
    "Write a Python function that converts a list into a dictionary where list elements are keys and their indices are values."
    
    Run the following sample input through the candidate’s code:
        sample_list = ['a', 'b', 'c']
    
    Display:
    - The Python function (if provided)
    - The result of calling the function with the sample input
    - A short explanation of what the function did
    
    Important:
    - If the function has syntax or logic errors, display the error message.
    - Be concise, but informative. You are helping validate the code's correctness.
    """
)

#agent4

reviewer_agent= AssistantAgent(
    name="reviewer_agent",
     llm_config=llm_config,
    system_message="""
    You are a Review Agent responsible for validating the correctness and quality of responses provided by an Answer Agent in a technical coding interview setting.

    Your tasks include:
    1. Reviewing the Python function and its execution output.
    2. Ensuring the function satisfies the original prompt: 
       "Convert a list into a dictionary where list elements are keys and their indices are values."
    3. Providing clear feedback on:
       - Correctness of the logic
       - Output accuracy
       - Code quality (readability, structure, Pythonic style)
    4. Identifying any issues, inefficiencies, or missed edge cases (like duplicates or empty lists), if applicable.
    5. Giving a final verdict: ✅ Pass / ❌ Needs Improvement
    
    Context:
    - You receive the candidate’s function, the test input (e.g., `['a', 'b', 'c']`), and the resulting output.
    - You may also receive a brief explanation from the Answer Agent.
    
    Your review should be structured as:
    - **Function Review**
    - **Output Review**
    - **Explanation Quality**
    - **Final Verdict**

    """
)
#agent 5

conversion_agent=AssistantAgent(
    name="conversion_agent",
     llm_config=llm_config,
    system_message="""
    You are a Conversion Agent tasked with converting verified and reviewed Python code into equivalent JavaScript.
    
    Context:
    - The Reviewer Agent has confirmed the Python function is correct.
    - The function converts a list into a dictionary where each list element becomes a key and its index becomes the corresponding value.
    
    Your tasks:
    1. Translate the Python function into clean, idiomatic JavaScript.
    2. Preserve the core logic and structure.
    3. Use appropriate JavaScript data structures (e.g., `Object` or `Map`).
    4. Include the function definition, sample input, and output.
    5. Add inline comments where helpful for clarity.
    
    Respond with:
    - **JavaScript Code Block**
    - **Sample Input & Output**
    - **Optional Explanation** (brief, especially if there are language-specific differences like object key constraints)
    
    Assume the original Python function was:
    ```python
    def list_to_dict(lst):
        return {item: index for index, item in enumerate(lst)}

    """
)
#agent 6

final_output=AssistantAgent(
    name="final_output",
     llm_config=llm_config,
    system_message="""
    You are the Final Agent responsible for validating that the logic and behavior of a Python function and its JavaScript equivalent are consistent.
    
    Your tasks:
    1. Compare the Python and JavaScript functions provided by the Conversion Agent.
    2. Verify both functions produce the **same output** for the **same input** (e.g., ['a', 'b', 'c'] → {'a': 0, 'b': 1, 'c': 2}).
    3. Evaluate:
       - Functional equivalence (same logic and result)
       - Syntactic differences due to language (acceptable)
       - Edge case handling (e.g., empty lists, duplicate keys)
    4. Point out any discrepancies, improvements, or edge case considerations.
    5. Provide a **final verdict**:
       - ✅ Equivalent
       - ⚠️ Partially Equivalent (explain what differs)
       - ❌ Not Equivalent (explain critical differences)
    
    Structure your response as:
    - **Logic Comparison**
    - **Output Comparison**
    - **Language-Specific Notes**
    - **Final Verdict**

    """
)


In [50]:
group_chat=GroupChat(
    agents=[
        user_agent,
        interviewer_agent,
        answer_agent,
        reviewer_agent,
        conversion_agent,
        final_output
    ],
    messages=[],
    max_round=10
)

manager=GroupChatManager(
    groupchat=group_chat,llm_config=llm_config
    
)

In [51]:
response= manager.initiate_chat(
    user_agent,
    message="Write a Python function that converts a list into a dictionary where each element is a key and its index is the value.",
    auto_reply=True
)

chat_manager (to user_agent):

Write a Python function that converts a list into a dictionary where each element is a key and its index is the value.

--------------------------------------------------------------------------------
user_agent (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: interviewer_agent

interviewer_agent (to chat_manager):

"Write a Python function that converts a list into a dictionary. Each element of the list should be used as a key in the dictionary, and the value can be the index of the element in the list."

--------------------------------------------------------------------------------

Next speaker: answer_agent

answer_agent (to chat_manager):

Sample function:
```python
def list_to_dict(input_list):
    return {value: index for index, value in enumerate(input_list)}
```

Executing the function with the sample input `sample_list = ['a', 'b', 'c']`:
```python
sample_list = ['a', 'b', 'c